<a href="https://colab.research.google.com/github/avk1943/Soccer-Event-Prediction/blob/main/LLM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RUN THIS CODE FILE ON TPU TO AVOID MEMORY ERRORS!!!

# Import Libraries

Loading necessary libraries and mounting to google drive

In [ ]:
import numpy as np
import pandas as pd;
import sklearn as sk
import os
import random;
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
import os;
print(os.getcwd())
from os import listdir
from os.path import isfile, join
from torchvision.datasets.utils import download_url
from zipfile import ZipFile


import torch
import os
import random
from numpy import linalg
from scipy import stats
from tqdm import tqdm
import torch.optim as optim
torch.set_printoptions(precision=4,sci_mode=False)
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F
from torch import optim
import sys
import io
!pip install py7zr
import py7zr
!pip install gdown

/content
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 48.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device('cpu')
print("Using {}.".format(device))

Using cpu.


### Downloading relevent datasets from google drive

In [ ]:
!gdown "https://drive.google.com/file/d/1-3DDMv-yRn8eCni8mvWQBhdsvp880M79&confirm=t"
#!gdown 1-3DDMv-yRn8eCni8mvWQBhdsvp880M79
#!gdown 1-3DDMv-yRn8eCni8mvWQBhdsvp880M79
df = pd.read_pickle("/content/drive/Shared drives/Soccer Event Prediction/fe_data.pkl")
df = df.reset_index()

/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:35: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/1-3DDMv-yRn8eCni8mvWQBhdsvp880M79&confirm=t
To: /content/1-3DDMv-yRn8eCni8mvWQBhdsvp880M79&confirm=t
3.04kB [00:00, 7.27MB/s]


In [ ]:
!gdown "https://drive.google.com/file/d/1opUX9mqzZDTAE0XQmuz0aobdLc2GS5Cv&content=t"
#!gdown 1opUX9mqzZDTAE0XQmuz0aobdLc2GS5Cv
#!gdown 1Jf3c4q6oXL5OzshFaTQ9lwQdhpzEJ_Kc    #Google Drive open access sharing link
teams_list = pd.read_csv("/content/drive/Shared drives/Soccer Event Prediction/soccer_teams_to_import.csv")

/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:35: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/1opUX9mqzZDTAE0XQmuz0aobdLc2GS5Cv&content=t
To: /content/1opUX9mqzZDTAE0XQmuz0aobdLc2GS5Cv&content=t
3.04kB [00:00, 9.33MB/s]


In [ ]:
################################################################################################
# 1b) filter dataframe: keep only matches of the teams of interest
################################################################################################
idx = np.repeat(False,len(df))
for i in range(len(teams_list)):
  idx0 = np.logical_and(df['comp'] == teams_list.loc[i,'Comp'],df['TID'] == teams_list.loc[i,'wyId'])
  idx = np.logical_or(idx,idx0)
df = df.loc[idx,:]

In [ ]:
################################################################################################
# 1c) choose a pool of matches from each team (s.t. we have just the right number i.e. number of trn + val + tst)
################################################################################################
idx = np.repeat(False,len(df))
np.random.seed(123)
for i in range(len(teams_list)):
  tempteam = teams_list.loc[i,:]
  tempmatches = np.unique(df.loc[df["TID"]==tempteam['wyId'],"MID"])
  num_matches_to_choose = tempteam['trn'] + tempteam['val'] + tempteam['tst']
  matches_to_keep = np.random.choice(tempmatches,num_matches_to_choose,replace=False) # random (seeded) choice of matches to choose
  idx0 = np.logical_and(df['comp'] == tempteam['Comp'],df['TID'] == tempteam['wyId'])
  idx1 = np.logical_and(idx0,np.isin(df['MID'],matches_to_keep))
  idx = np.logical_or(idx,idx1)

In [ ]:
################################################################################################
# 1d) filter dataframe further now we have chosen the matches we want
################################################################################################
df = df.loc[idx,:]
df = df.sort_values(by=['TID','MID','index'])
df = df.drop("index",axis=1)
df = df.reset_index(drop=True)
df = df.reset_index(drop=False)

In [ ]:
################################################################################################
# 1e) now assign specific matches from teams_list against each team
################################################################################################
#(training/validation), and (test)
teams_list['trn_MIDs'], teams_list['val_MIDs'], teams_list['tst_MIDs'] = None, None, None
teams_list['n_trn'], teams_list['n_val'], teams_list['n_tst'] = 0, 0, 0
teams_list['trn_MIDs'], teams_list['val_MIDs'], teams_list['tst_MIDs'] = teams_list['trn_MIDs'].astype(object), teams_list['val_MIDs'].astype(object), teams_list['tst_MIDs'].astype(object)

np.random.seed(123)
for i in range(len(teams_list)):
  tempteam = teams_list.loc[i,:]
  tempteam_allmatches = np.unique(df.loc[df["TID"] == tempteam["wyId"],"MID"])
  np.random.shuffle(tempteam_allmatches)                       # perturbation: shuffle all of this team's MIDs in place
  teams_list['trn_MIDs'][i] = tempteam_allmatches[:tempteam["trn"]]
  teams_list['val_MIDs'][i] = tempteam_allmatches[tempteam["trn"]:(tempteam["trn"]+tempteam["val"])]
  teams_list['tst_MIDs'][i] = tempteam_allmatches[(tempteam["trn"]+tempteam["val"]):]
  teams_list['n_trn'][i] = np.sum(np.isin(df["MID"],teams_list['trn_MIDs'][i]))
  teams_list['n_val'][i] = np.sum(np.isin(df["MID"],teams_list['val_MIDs'][i]))
  teams_list['n_tst'][i] = np.sum(np.isin(df["MID"],teams_list['tst_MIDs'][i]))

teams_list

<ipython-input-11-0f129a58c97e>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teams_list['trn_MIDs'][i] = tempteam_allmatches[:tempteam["trn"]]
<ipython-input-11-0f129a58c97e>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teams_list['val_MIDs'][i] = tempteam_allmatches[tempteam["trn"]:(tempteam["trn"]+tempteam["val"])]
<ipython-input-11-0f129a58c97e>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teams_list['tst_MIDs'][i] = 

,Comp,Pos,Team,trn,val,tst,wyId,trn_MIDs,val_MIDs,tst_MIDs,n_trn,n_val,n_tst
0,EN,1,Manchester City,3,0,0,1625,"[2499781, 2499822, 2499841]",[],[],2800,0,0
1,EN,2,Manchester United,0,0,3,1611,[],[],"[2499914, 2499898, 2500001]",0,0,2668
2,EN,10,Newcastle United,0,0,3,1613,[],[],"[2499964, 2500027, 2499823]",0,0,1867
3,EN,11,Crystal Palace,3,1,0,1628,"[2499741, 2499961, 2499774]",[2499894],[],1921,670,0
4,EN,19,Stoke City,0,0,3,1639,[],[],"[2499765, 2499805, 2499920]",0,0,1997
5,EN,20,West Bromwich Albion,3,0,0,1627,"[2500019, 2499792, 2499854]",[],[],1818,0,0
6,DE,1,Bayern Munich,3,0,0,2444,"[2517018, 2516982, 2516997]",[],[],2623,0,0
7,DE,2,Schalke 04,0,0,3,2449,[],[],"[2517011, 2516770, 2516797]",0,0,1929
8,DE,9,Borussia Mönchengladbach,0,0,3,2454,[],[],"[2516851, 2516778, 2516814]",0,0,2062
9,DE,10,Hertha BSC,3,1,0,2457,"[2516967, 2516840, 2516813]",[2516920],[],2226,669,0


In [ ]:
################################################################################################
# 2) Insert match buffer after each match
################################################################################################

paddingrow = df.iloc[-1,:]
paddingrow = pd.DataFrame(paddingrow).T
paddingrow["comp"] = "na"       #need to update each time padding is added
paddingrow["TID"] = 123         #need to update each time padding is added
paddingrow["MID"] = 123         #need to update each time padding is added
paddingrow["index"] = 1000000000
paddingrow["PID"] = 1000000000
paddingrow["act"] = "@"
paddingrow["T"] = 0.5           #df["T"].mean()
paddingrow["deltaT"] = 0.5      #df["deltaT"].mean()
paddingrow["x"] = 0.5          #df["x"].mean()
paddingrow["y"] = 0.5          #df["y"].mean()
paddingrow["s"] = 0.2           #df["s"].mean()
paddingrow["scrad"] = 0.5       #df["scrad"].mean()
paddingrow["deltax"] = 0.5     #df["deltax"].mean()
paddingrow["deltay"] = 0.5     #df["deltay"].mean()
paddingrow["sg"] = 0.57         #df["sg"].mean()
paddingrow["thetag"] = 0.74     #df["thetag"].mean()
num_paddingrows=1
paddingrows = paddingrow.iloc[np.repeat(0,num_paddingrows),:]

################################################################################################
# 2b) make a generic buffer row (we'll need to update the '123' fields however before adding in the for loop)
################################################################################################
def insert_rows(idx, df, df_to_insert):  # convenience function to insert a row
    dfA = df.iloc[:idx, ]
    dfB = df.iloc[idx:, ]
    df = dfA.append(df_to_insert).append(dfB).reset_index(drop = True)
    return df

################################################################################################
# 2c) find out where we need to insert this padding/buffers
################################################################################################
bufidxns = np.where(df["MID"][1:].to_numpy() != df["MID"][:-1].to_numpy())[0]  #the idxn indicates the LAST row of a match (MID)

################################################################################################
# 2d) insert the buffers
################################################################################################
for i in tqdm(range(0,len(bufidxns))):   # insert the padding/buffers
  # print(i)
  bufidxns = np.where(df["MID"][1:].to_numpy() != df["MID"][:-1].to_numpy())[0]  #recompute each time...
  bufidxn = bufidxns[i]
  paddingrows["comp"] = df.loc[bufidxn,"comp"]
  paddingrows["TID"] = df.loc[bufidxn,"TID"]
  paddingrows["MID"] = df.loc[bufidxn,"MID"]
  df = insert_rows(bufidxn+1,df,paddingrows)

df = df.drop("index",axis=1)
df = df.reset_index(drop=True)
df = df.reset_index(drop=False)

##########################################################################################
# 2e) minor tidy-up: rearrange columns
##########################################################################################
df = df.loc[:,['comp', 'TID', 'MID', 'index', 'PID', 'act', 'T', 'x', 'y', 'sg', 'thetag', 'scrad', 'deltaT', 'deltax', 'deltay', 's']]

  0%|          | 0/144 [00:00<?, ?it/s]<ipython-input-12-d892d6ed4287>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = dfA.append(df_to_insert).append(dfB).reset_index(drop = True)
<ipython-input-12-d892d6ed4287>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = dfA.append(df_to_insert).append(dfB).reset_index(drop = True)
<ipython-input-12-d892d6ed4287>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = dfA.append(df_to_insert).append(dfB).reset_index(drop = True)
<ipython-input-12-d892d6ed4287>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = dfA.append(df_to_insert).append(dfB).reset_index(drop = True)
<ipython-

In [ ]:
##########################################################################################
# 3) Define data loaders
##########################################################################################

##########################################################################################
# 3a) Specify indices for whole, trnval, tst, dmo
##########################################################################################
np.random.seed(123)
##########################################################################################
# 3a)i)   whole set -> idx_all (100% of total)
##########################################################################################
idx_all = np.repeat(True,len(df))
print("n idx_all",np.sum(idx_all))

##########################################################################################
# 3a)ii)  trn set   -> idx_trn
##########################################################################################
idx_trn = np.repeat(False,len(df))
for i in range(len(teams_list)):
  tempteam = teams_list.loc[i,:]
  idx0 = np.logical_and(df["TID"]==tempteam["wyId"],np.isin(df["MID"],tempteam['trn_MIDs']))
  idx_trn = np.logical_or(idx_trn,idx0)
print("n idx_trn",np.sum(idx_trn))

##########################################################################################
# 3a)iii) val set   -> idx_val
##########################################################################################
idx_val = np.repeat(False,len(df))
for i in range(len(teams_list)):
  tempteam = teams_list.loc[i,:]
  idx0 = np.logical_and(df["TID"]==tempteam["wyId"],np.isin(df["MID"],tempteam['val_MIDs']))
  idx_val = np.logical_or(idx_val,idx0)
print("n idx_val",np.sum(idx_val))

##########################################################################################
# 3a)iv)  tst set   -> idx_tst
##########################################################################################
idx_tst = np.repeat(False,len(df))
for i in range(len(teams_list)):
  tempteam = teams_list.loc[i,:]
  idx0 = np.logical_and(df["TID"]==tempteam["wyId"],np.isin(df["MID"],tempteam['tst_MIDs']))
  idx_tst = np.logical_or(idx_tst,idx0)
print("n idx_tst",np.sum(idx_tst))

##########################################################################################
# 3a)v)   dmo set   -> idx_dmo
##########################################################################################
# 2 matches in demo set
# MID 2058017, France - Croatia, 4 - 2, 15 July 2018
# MID 2576028, Udinese - Sampdoria, 4 - 0, 30 Sept 2017
idx_dmo = np.isin(df["MID"],np.array([2058017,2576028]))
print("n idx_dmo",np.sum(idx_dmo))


##########################################################################################
# 3a)vii) summary of MIDs used
##########################################################################################
print("idx_trn:\n", np.unique(df.loc[idx_trn,"MID"].map(str) + "-" + df.loc[idx_trn,"TID"].map(str)))
print("idx_val:\n", np.unique(df.loc[idx_val,"MID"].map(str) + "-" + df.loc[idx_val,"TID"].map(str)))
print("idx_tst:\n", np.unique(df.loc[idx_tst,"MID"].map(str) + "-" + df.loc[idx_tst,"TID"].map(str)))
print("idx_dmo:\n", np.unique(df.loc[idx_dmo,"MID"].map(str) + "-" + df.loc[idx_dmo,"TID"].map(str)))

n idx_all 101895
n idx_trn 48463
n idx_val 4932
n idx_tst 48500
n idx_dmo 1332
idx_trn:
 ['1694409-11555' '1694410-11555' '1694413-11555' '1694417-8274'
 '1694418-8274' '1694421-9905' '1694422-10451' '1694423-9905'
 '1694424-10451' '1694429-8274' '1694431-10451' '1694440-9905'
 '2057968-4418' '2057985-14855' '2057987-14855' '2057988-14855'
 '2057990-17929' '2057993-17929' '2057995-17929' '2057998-12430'
 '2058001-12430' '2058009-12430' '2058014-4418' '2058017-4418'
 '2499741-1628' '2499774-1628' '2499781-1625' '2499792-1627'
 '2499822-1625' '2499841-1625' '2499854-1627' '2499961-1628'
 '2500019-1627' '2500699-3804' '2500859-3777' '2500904-3804'
 '2500947-3804' '2500952-3767' '2500972-3767' '2500983-3767'
 '2501001-3777' '2501060-3777' '2516762-2463' '2516813-2457'
 '2516831-2463' '2516840-2457' '2516844-2463' '2516967-2457'
 '2516982-2444' '2516997-2444' '2517018-2444' '2565560-691' '2565576-683'
 '2565681-676' '2565702-683' '2565704-676' '2565734-683' '2565786-691'
 '2565820-676' '256

In [ ]:
##########################################################################################
# 3b) Define valid slices
##########################################################################################

##########################################################################################
# 3b)i) Hyperparameters
##########################################################################################
maxlen = 40
step = 1

##########################################################################################
# 3b)ii) Define valid slices
##########################################################################################
df["valid_slice_flag"]=False
matches = np.unique(df["MID"])
for m in matches:
  #print(m)
  idx = df["MID"] == m
  idxn = np.where(idx)[0]
  df.loc[idxn[maxlen::step],"valid_slice_flag"] = True     #the first slice will be start at "maxlen" into match; and then be offset by "step" until end of match
  df.loc[idxn[-1],"valid_slice_flag"] = False              #exclude the final row, which is the sample corresponding to EOM "@" action

In [ ]:
##########################################################################################
# 3c) Define PyTorch dataset class and specific datasets instances and associated dataloaders
##########################################################################################

##########################################################################################
# 3c)i)   Define dataset class
##########################################################################################
class SoccerDataset(Dataset):
    # cut the text in semi-redundant sequences of maxlen characters
    def __init__(self,idx=idx_all):
        self.idx = idx
        self.valid_slice_idxn = np.where(np.logical_and(self.idx,df["valid_slice_flag"]))[0]  #in both the idx and has a valid slice flag
    def __len__(self):
        return int(np.sum(df.loc[self.idx,"valid_slice_flag"]))
    def __getitem__(self, i):
        j = self.valid_slice_idxn[i]
        y = df.iloc[j].loc[target_vars]
        x = df.iloc[(j-maxlen):j].loc[:,input_vars]
        x = x.to_numpy().astype(float)
        y = y.to_numpy().astype(float)
        sample = {'x': x, 'y': y}
        return sample
        #return x.to_numpy(), y.to_numpy().astype(float)
    def to_dataset(self):
        return torch.utils.data.TensorDataset(self.data)
    def map(self, func):
        return torch.data.Dataset.from_tensor_slices(list(map(func, self._data)))

##########################################################################################
# 3c)ii)  Define datasets
##########################################################################################
# define datasets
all_dataset = SoccerDataset()
trn_dataset = SoccerDataset(idx=idx_trn)
val_dataset = SoccerDataset(idx=idx_val)
tst_dataset = SoccerDataset(idx=idx_tst)
dmo_dataset = SoccerDataset(idx=idx_dmo)

##########################################################################################
# 3c)iii) Define dataloaders
##########################################################################################
batch_size, num_workers = 10, 2
all_loader = DataLoader(all_dataset,shuffle=False,batch_size=batch_size,num_workers=num_workers,drop_last=True)
trn_loader = DataLoader(trn_dataset,shuffle=True,batch_size=batch_size,num_workers=num_workers,drop_last=True)
val_loader = DataLoader(val_dataset,shuffle=False,batch_size=batch_size,num_workers=num_workers,drop_last=True)
tst_loader = DataLoader(tst_dataset,shuffle=False,batch_size=batch_size,num_workers=num_workers,drop_last=True)
dmo_loader = DataLoader(dmo_dataset,shuffle=False,batch_size=1,num_workers=num_workers)

In [ ]:
##########################################################################################
# 4) Define categorical action encode/decode
##########################################################################################
# replace own goal "h" with goal "g" (rare instance... simplest solution for this version... to do 'properly' in future development...)
idx = df["act"] == "h"
df.loc[idx,"act"] = "g"

# set up idx2char and char2idx
chars = sorted(list(set(df['act'])))     # list of all unique characters in the text
num_chars_test = len(chars)
char2idx_test = dict((c, i) for i, c in enumerate(chars))
idx2char_test = dict((i, c) for i, c in enumerate(chars))

num_chars=7 # hyperparameter
idx2char = {0: '@', 1: '_', 2: 'd', 3: 'g', 4: 'p', 5: 's', 6: 'x'}
char2idx = {'@': 0, '_': 1, 'd': 2, 'g': 3, 'p': 4, 's': 5, 'x': 6}

# let's just check that our hard coded values match what's in the data at this point...
assert num_chars_test == num_chars
#assert idx2char_test == idx2char
#assert char2idx_test == char2idx

# replace characters with numbers
df['act'].replace(char2idx,inplace=True)
print(idx2char)

##########################################################################################
# 5) Specify variables of interest
##########################################################################################
input_vars  = ['act', 'T', 'x', 'y', 'sg', 'thetag', 'scrad', 'deltaT', 'deltax', 'deltay', 's']
target_vars = ['act', 'x', 'y']

#df.columns: ['comp', 'TID', 'MID', 'index', 'PID', 'act', 'T', 'x', 'y', 'sg', 'thetag', 'scrad', 'deltaT', 'deltax', 'deltay', 's']

##########################################################################################
# 6) Specify loss function weighting
##########################################################################################
##########################################################################################
# 6a) Action, categorical
##########################################################################################
print(idx2char)
print(np.unique(df["act"],return_counts=True))
weights = char2idx.copy()
for x in weights: weights[x]=1
weights['@'] = 0
weights['_'] = 0
weights['d'] = 0.120  #10  #0.1 #7826/78659
weights['g'] = 0
weights['p'] = 0.013  #1.2 #0.84 #66026/78659
weights['s'] = 0.580  #50  #0.02 #1653/78659
weights['x'] = 0.29   #25  #0.04 #3154
print(weights)               #we are more accepting if the model doesn't predict low prob events like goals
weights = torch.tensor(list(weights.values()))
print(weights)

##########################################################################################
# 6b) All other variables, continuous
##########################################################################################
cont_weights = torch.tensor([1.,1.])
print(target_vars[1:])
print(cont_weights)

{0: '@', 1: '_', 2: 'd', 3: 'g', 4: 'p', 5: 's', 6: 'x'}
{0: '@', 1: '_', 2: 'd', 3: 'g', 4: 'p', 5: 's', 6: 'x'}
(array([0, 1, 2, 3, 4, 5, 6]), array([  144, 18882,  8201,   192, 69422,  1742,  3312]))
{'@': 0, '_': 0, 'd': 0.12, 'g': 0, 'p': 0.013, 's': 0.58, 'x': 0.29}
tensor([0.0000, 0.0000, 0.1200, 0.0000, 0.0130, 0.5800, 0.2900])
['x', 'y']
tensor([1., 1.])


In [ ]:
for batch in trn_loader:
    print(batch['x'].shape)
    print(batch['y'].shape)
    break

torch.Size([10, 40, 11])
torch.Size([10, 3])


# FLAN-T5 Model Training

In [ ]:
!pip install nltk
!pip install datasets
!pip install transformers[torch]
!pip install tokenizers
!pip install evaluate
!pip install rouge_score
!pip install transformers[sentencepiece]
!pip install huggingface_hub

In [ ]:
!pip install torchdata

In [ ]:
import nltk
#import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
!pip install sentencepiece

In [ ]:
MODEL_NAME = "google/flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer

In [ ]:
MODEL_NAME = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
#tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name, trust_remote_code=True)

In [ ]:
prefix = "Please predict the next possible soccer event action & position after this: "

# Define the preprocessing function

def preprocess_function(examples):
    """Add prefix to the sentences, tokenize the text, and set the labels"""
    # The "inputs" are the tokenized answer:
    inputs = examples["x"]
    model_inputs = tokenizer(inputs, max_length=10, truncation=True)

    # The "labels" are the tokenized outputs:
    #labels = tokenizer(text_target=examples["y"], max_length=40, truncation=True)
    labels = examples["y"]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
def tokenize_func(examples):
	return tokenizer(examples["x"], examples["y"], truncation=True)  # max_length=512,  padding=True

In [ ]:
from torchdata.datapipes.map import Mapper

In [ ]:
tokenized_train_dataset = Mapper(trn_dataset, preprocess_function)
#tokenized_train_dataset = Mapper(trn_dataset, tokenize_func)

In [ ]:
tokenized_test_dataset = Mapper(tst_dataset, preprocess_function)
#tokenized_test_dataset = Mapper(tst_dataset, tokenize_func)

In [ ]:
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 10
PER_DEVICE_EVAL_BATCH = 4
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 3

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=L_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
    weight_decay=WEIGHT_DECAY,
    save_total_limit=SAVE_TOTAL_LIM,
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    push_to_hub=False
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    #compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

# LLM Model Training
Using the Salesforce XGen-7b LLM

Installing the necessary libaries

CAUTION: Do not change the order of the installation

In [ ]:
!pip install trl
!pip install peft
!pip install transformers
!pip install datasets
!pip install tiktoken

In [ ]:
import torch
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments

In [ ]:
!pip install trl --upgrade
!pip install -U accelerate
!pip install -U transformers

In [ ]:
pretrained_model_name = "Salesforce/xgen-7b-8k-base"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name, torch_dtype=torch.bfloat16)
#tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/510 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.68G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/329 [00:00<?, ?B/s]

tokenization_xgen.py:   0%|          | 0.00/8.85k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Salesforce/xgen-7b-8k-base:
- tokenization_xgen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [ ]:
model_training_args = TrainingArguments(
       output_dir="xgen-7b-8k-base-fine-tuned",
       per_device_train_batch_size=2,
       optim="adamw_torch",
       logging_steps=5,
       learning_rate=2e-4,
       warmup_ratio=0.1,
       lr_scheduler_type="linear",
       num_train_epochs=1,
       save_strategy="epoch"
   )

In [ ]:
lora_peft_config = LoraConfig(r=16, lora_alpha=32, lora_dropout=0.05, task_type="CAUSAL_LM")

In [ ]:
SFT_trainer = SFTTrainer(
       model=model,
       train_dataset=trn_dataset,
       dataset_text_field="text",
       max_seq_length=10,
       tokenizer=tokenizer,
       args=model_training_args,
       packing=True,
       #data_generator=data_generator,
       #optimizer_grouped_parameters=model.parameters(),
       peft_config=lora_peft_config,
   )

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))
model = prepare_model_for_int8_training(model)
model = get_peft_model(model, lora_peft_config)


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:141: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [ ]:
training_args = TrainingArguments(**model_training_args.to_dict())
#training_args = model_training_args
trainer = SFT_trainer
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1751: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(


AttributeError: ignored